In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
pip install datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 924.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from evaluate import load as load_metric
import logging
import os
import re

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
MODELS = {
    "UI-TARS 7B SFT": "bytedance-research/UI-TARS-7B-DPO"
}

In [ ]:
def load_model(model_path):
    """Load model and tokenizer with Hugging Face authentication."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hf_token = os.getenv("HUGGINGFACE_TOKEN")

    """Load model and tokenizer."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.empty_cache()
    model = Qwen2VLForConditionalGeneration.from_pretrained(

        model_path,
        torch_dtype=torch.float16,
        device_map={"": 0},
        offload_folder="offload",
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        token=hf_token
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path, token=hf_token)
    logger.info(f"{model_path} Model Loaded Successfully!")
    return model, tokenizer

In [ ]:
def evaluate_math(model, tokenizer, dataset_name="gsm8k"):
    """Evaluate on GSM8K for Math QA and display actual vs. predicted answers."""
    dataset = load_dataset(dataset_name, 'main', split="test[:5]")
    metric = load_metric("accuracy")

    for example in dataset:
        input_text = f"Solve: {example['question']}"
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        pred_numbers = re.findall(r"[-+]?\d*\.\d+|\d+", prediction)
        answer_numbers = re.findall(r"[-+]?\d*\.\d+|\d+", str(example['answer']))

        if pred_numbers:
            prediction = pred_numbers[-1]
        if answer_numbers:
            answer = answer_numbers[-1]
        else:
            answer = "0"

        try:
            prediction = float(prediction) if '.' in prediction else int(prediction)
            answer = float(answer) if '.' in answer else int(answer)
        except ValueError:
            prediction, answer = 0, 0

        metric.add(prediction=prediction, reference=answer)

        print("\n📝 Question:", example['question'])
        print("✅ Ground Truth Answer:", example['answer'])
        print("🤖 Model Prediction:", prediction)

    return metric.compute()

In [ ]:
for model_name, model_path in MODELS.items():
    print(f"\nEvaluating {model_name}...")
    model, tokenizer = load_model(model_path)

    print("GSM8K Math:", evaluate_math(model, tokenizer))


Evaluating UI-TARS 7B SFT...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]


📝 Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
✅ Ground Truth Answer: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
🤖 Model Prediction: 18

📝 Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
✅ Ground Truth Answer: It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric
#### 3
🤖 Model Prediction: 1

📝 Question: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?
✅ Ground Truth Answer: The cost of the house and repairs